# ✅**Contents_Based(완료).ipynb - 문서 유사도 판단** 
---
### 5만개의 Document Clustering 결과를 사용해 유사도가 높은 문서 5개를(미정) 사용자에게 추천(contents-based)

*   각 cluster의 feature와 target의 feature를 count해 가장 유사한 clusters를 찾음
*   찾은 clusters 안의 기사들에 대해 target과 비교 후 상위 5개의 기사를 추천 

> Used Method, Skills 
* Data PreProcessing
* TF-IDF Based Document Similarity


In [ ]:
%%bash 
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy
%env JAVA_HOME "usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import re, csv
pd.options.mode.chained_assignment = None

import numpy as np
np.random.seed(0)

from konlpy.tag import Okt
okt = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [ ]:
# 불용어(Stop_words) 
with open('/content/drive/MyDrive/DataMining TeamProject/stopword.txt', 'r') as f:
    STOP_WORDS = f.read().replace('\n',' ').split()  

# Clustering 결과를 Read
cluster_features = pd.read_csv('/content/drive/MyDrive/DataMining TeamProject/clustering_feature.csv', encoding='euc-kr', header=None)
cluster_news = pd.read_csv('/content/drive/MyDrive/DataMining TeamProject/clustering_news.csv', encoding='euc-kr', header=None)

news_list = cluster_news.values.tolist()
cluster_features.columns = ["num", "feature"]
feature_list = cluster_features['feature'].tolist()

In [ ]:
def tokenizer(raw, pos=["Noun", "Alpha", "Verb"], stopword=STOP_WORDS):
    return [
        word for word, tag in okt.pos(raw, norm=True, stem=True)
        if len(word) > 1 and tag in pos and word not in stopword
    ]

def get_similar_clusters(feature_list, target):
    temp = []
    for idx, features in enumerate(feature_list):
        count = 0
        for feature in features[1:-1].replace(',', '').replace("'",'').split():
            count += target.count(feature)
        temp.append((str(idx), count))
    return [num[0] for num in sorted(temp, key=lambda x:-x[1])[:3]]
            
def get_recommend_news(similar_clusters, target):
    rawdata = []
    for news in news_list:
        if news[-1] in similar_clusters:
            rawdata.append(news[2])
    
    vectorize = TfidfVectorizer(tokenizer = tokenizer, min_df=2, sublinear_tf = True)
    X = vectorize.fit_transform(rawdata)
    features = vectorize.get_feature_names_out()

    srch = [t for t in tokenizer(target) if t in features]

    srch_dtm = np.asarray(X.toarray())[:, [
    vectorize.vocabulary_.get(i) for i in srch                                                        
    ]]

    result = {}
    score = srch_dtm.sum(axis=1)
    for i in score.argsort()[::-1]:
        if score[i] > 0:
            result[score[i]] = rawdata[i]
    
    for key, value in list(result.items())[:5]:
        print(f'{key} : {value}')

target = 
(   
    "Riot 아케인 이벤트 대표 이미지 라이엇게임즈 제공 지난 달간 리그오브레전드 LoL 유니버스 게임의 월간 사용자 MAU 억 기록했다 라이엇게임즈는 내용을 밝히면서 역대 최고 기록이라고 설명했다 
    "지난 년간 리그오브레전드 유니버스 게임을 즐긴 세계 플레이어는 억 명에 달했다 니콜로 러렌트 라이엇게임즈 CEO는 때보다 게임을 즐기는 이들이 많아진 애니메이션 아케인을 통해 세계 각지
    "플레이어와 손잡고 게임을 글로벌 엔터테인먼트 산업의 중심으로 자리 잡게 하고자 한다 고 했다 아케인 은 리그오브레전드 지식재산권 기반 첫 번째 장편 애니메이션이다 오는 롤드컵 이후 
    "넷플릭스에서 세계 동시 공개된다 첫 화는 트위치에서도 동시 중계로 감상할 라이엇게임즈는 아케인 출시를 기념해 발로란트 와일드 리프트 전략적 팀 전투 자사 모든 게임에 적용하는 Riot 
    "아케인 이벤트 한다 게임별 이벤트는 애니메이션 주제와 내용을 테마로 신규 콘텐츠 등에 초점을 맞췄다 라이엇게임즈는 롤드컵 개막식 날인 오는 오전 시 분 아케인 애니메이션을 처음 
    "글로벌 프리미어 행사 미국 로스앤젤레스에 위치한 라이엇게임즈 본사에서 한다 사라 슈츠 라이엇게임즈 익스피리언스 부문 책임자는 우리는 리그오브레전드 지식재산권의 기원인 게임을 아케인과
    "관련된 경험의 출발점으로 삼았다 고 했다"
)

target = ' '.join([t for t in list(set(okt.nouns(target))) if len(t) > 1 not in STOP_WORDS])

# target과 가장 비슷한 cluster
similar_clusters = get_similar_clusters(feature_list, target)

get_recommend_news(similar_clusters, target)